In [ ]:
#@title << - Click the play button to run.
#@markdown Double click here to view the code.

import json
from google.colab import files
from google_auth_oauthlib.flow import Flow

# Upload client secrets file
print('Upload your client_secret.json file:\n')
uploaded = files.upload()
uploaded_file_name = list(uploaded)[0]

if not uploaded_file_name.startswith('client_secret'): 
    raise Exception(f"Invaild file: {uploaded_file_name}, Please upload the client_secret.json file.")

# Create the flow using the client secrets file from the Google API
# Console.
flow = Flow.from_client_secrets_file(
    uploaded_file_name,
    scopes=['https://www.googleapis.com/auth/drive'],
    redirect_uri='urn:ietf:wg:oauth:2.0:oob')

# Tell the user to go to the authorization URL.
auth_url, _ = flow.authorization_url(prompt='consent')

print(f'\nPlease go to this URL: {auth_url}')

# The user will get an authorization code. This code is used to get the
# access token.
code = input('Enter the authorization code: ')
flow.fetch_token(code=code)

# Gets token json as a string 
token_json = json.dumps(
    {
        'token': flow.credentials.token,
        'refresh_token': flow.credentials.refresh_token,
        'token_uri': flow.credentials.token_uri,
        'client_id': flow.credentials.client_id,
        'client_secret': flow.credentials.client_secret,
        'scopes': flow.credentials.scopes
    }
)

# Fetch the js code for cf proxy from the repo
def get_js():
    !curl -s "https://raw.githubusercontent.com/ssnjrthegr8/stremio-gdrive/main/cf_proxy.js" -O cf_proxy.js
    authconfig = f'var authConfig = {{"client_id": "{flow.credentials.client_id}","client_secret": "{flow.credentials.client_secret}","refresh_token": "{flow.credentials.refresh_token}"}};'
    with open('cf_proxy.js', 'r') as js:
        out = ''
        read = False
        for line in js.readlines():
            if line.startswith('addEventListener'):
                read = True
            if read:
                out += line
        return f'{authconfig}\n{out}'

# Prints the outputs
print(f'\n1. Copy the token string for heroku:\n\n{token_json}\n')
print(f'\n2. Copy the code for cloudflare workers:\n\n{get_js()}')